# Tratamento de Lista Geral de Leads

### Objetivo: Esse pipeline tem como objetivo separar a lista de Leads disponível para disparos e consumo

In [1]:
# Importações básicas
import pandas as pd
import numpy as np
import sys
from pathlib import Path
import os
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'src'))

# Adiciona src ao path
sys.path.append('../src')

#

# Utilitários de dados
from data_utils import (
    load_raw_data,
    save_processed_data,
    remove_duplicates,
    handle_missing_values,
    detect_outliers,
    normalize_column,
    process_phone_string,
    process_phone_number,
    clean_and_lower_column,
    flatten_list_to_df,
    remove_buyers_from_dataframe
)

CRONOGRAMA_SUBDOMAIN = 'cronogramadosfluentes-xwamel'

# Utilitários SQL
from sql_utils import DatabaseConnection as Dbc, load_query_from_file

# Utilitários de visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Utilitários de API
from api_utils import (
    make_request,
    get_json,
    post_json,
    paginated_request,
    response_to_dataframe
)

# utilitários hotmart
from hotmart_utils import Hotmart

# utilitários tmb
from tmb_utils import TMB   

# Configurações pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Load Database Driver
db = Dbc()

# Inicializar API Hotmart
hotmart = Hotmart()

# Inicializar API TMB
tmb = TMB()

print('✓ Importações concluídas com sucesso!')

✓ Importações concluídas com sucesso!


## Load Leads Dataframe

In [ ]:
df = db.execute_query_from_file("select_complete_lead_data_from_campaign", params={"campaign_id": "isca-aulassemanais", "conversion_type_id": 1})

In [ ]:
# Garante datetime correto
df["last_conversion_date"] = pd.to_datetime(df["last_conversion_date"], format="%Y-%m-%d", errors="coerce")

# Cria coluna de referência semanal, semana iniciando no domingo
df["conversion_week"] = df["last_conversion_date"].dt.to_period("W-SUN")
df["conversion_week_start"] = df["conversion_week"].dt.start_time

# Filtra as semanas do último mês de 2025 (dezembro) e a primeira semana de 2026
mask = (
    ((df["conversion_week_start"].dt.year == 2025) & (df["conversion_week_start"].dt.month == 12))
    | (
        (df["conversion_week_start"].dt.year == 2026)
        & (df["conversion_week_start"].dt.month == 1)
        & (df["conversion_week_start"].dt.isocalendar().week == 1)
    )
)
df_semanas_filtradas = df[mask].copy()

# Agrupamento e cálculos
agg_df = (
    df_semanas_filtradas.groupby("conversion_week_start")
    .agg(
        leads=("lead_id", "nunique"),
        multi_qualificados=("qualificado_geral", lambda x: df_semanas_filtradas.loc[x.index][x].lead_id.nunique()),
        renda_qualificada=("renda_qualificada", lambda x: df_semanas_filtradas.loc[x.index][x].lead_id.nunique()),
    )
    .reset_index()
)

# Calcula percentuais
agg_df["perc_multi_qualificados"] = (agg_df["multi_qualificados"] / agg_df["leads"]).round(4)
agg_df["perc_renda_qualificada"] = (agg_df["renda_qualificada"] / agg_df["leads"]).round(4)
agg_df

,conversion_week_start,leads,multi_qualificados,renda_qualificada,perc_multi_qualificados,perc_renda_qualificada
0,2025-12-01,7257,720,1500,0.0992,0.2067
1,2025-12-08,5282,583,1204,0.1104,0.2279
2,2025-12-15,2029,293,538,0.1444,0.2652
3,2025-12-22,2207,326,606,0.1477,0.2746
4,2025-12-29,1459,224,383,0.1535,0.2625


In [ ]:
## Filter Buyers from Dataframe
df = remove_buyers_from_dataframe(df)

✓ Removidos 1,242 compradores de 105,292 registros (1.2% filtrado)


In [ ]:
# garante datetime
df["last_conversion_date"] = pd.to_datetime(df["last_conversion_date"], format="%Y-%m-%d", errors="coerce")

cutoff = pd.Timestamp.now().normalize() - pd.Timedelta(days=7)
df_ultimos_8_dias = df[df["last_conversion_date"] >= cutoff].copy()